# EchoCare: Cry Classification Model Training
## Stage 2 of Two-Stage Pipeline

This notebook trains a binary classification model to detect pain cries vs hungry cries in infants.

**Dataset:**
- Baby Chillanto Dataset 

**Categories:**
- Pain
- Hungry

**Target Performance:** >70% accuracy for cry classification

**Architecture:** MobileNetV2 (lightweight for Raspberry Pi deployment) with custom classification head

## 1. Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.20.0
Keras version: 3.12.0


## 2. Configuration

In [2]:
# Paths
train_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_classification/train")
val_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_classification/validate")
test_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_classification/test")
save_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/model/cry_classification")

# Model hyperparameters
img_size = (224, 224)  # MobileNetV2 input size
batch_size = 32
epochs = 30
learning_rate = 0.0001
dropout_rate1 = 0.3
dropout_rate2 = 0.2

# Class information
class_names = ['pain', 'hungry']
num_classes = 2

## 3. Data Loading Functions

In [ ]:
def load_spectrograms(data_dir, verbose=True):
    """
    Load .npy spectrograms from directory structure.

    Args:
        data_dir: Path object or string path to directory containing pain/hungry folders
        verbose: Print loading progress
    
    Returns:
        spectrograms: numpy array of spectrograms
        labels: numpy array of labels (0=pain, 1=hungry)
    """

    spectrograms = []
    labels = []
    
    # Load hungry mel-spectrograms (label = 1)
    hungry_dir = data_dir / 'hungry'  # Find the 'hungry' subfolder
    hungry_files = list(hungry_dir.glob('*.npy')) # Get all .npy files in the 'hungry' folder
    
    for file in hungry_files: # Load each hungry file and label it as 1 (hungry = 1)
        spec = np.load(file) # load the .npy (mel spectrogram) file
        spectrograms.append(spec) # add to mel-spectrograms list
        labels.append(1) # label as hungry (1)
    
    if verbose:
        print(f"Loaded {len(hungry_files)} hungry spectrograms")
    
    # Load pain spectrograms (label = 0)
    pain_dir = data_dir / 'pain'
    pain_files = list(pain_dir.glob('*.npy'))
    
    for file in pain_files: # Load each pain file and label it as 0 (pain = 0)
        spec = np.load(file)
        spectrograms.append(spec)
        labels.append(0)
    
    if verbose:
        print(f"Loaded {len(pain_files)} pain spectrograms")
        print(f"Total samples: {len(spectrograms)}")
    
    return np.array(spectrograms), np.array(labels)